In [1]:
# Shera Adams
# shera.adams@snhu.edu
# January 23,2024
# 
# Version: 2.0, This is an enhanced version of the CS340 Client-server dashboard project created for Southern New Hampshire University to 
# satisfy the databases category enhancement of CS499, Capstone class.
#
# The main purpose of main.ipynb is to establish a connection to the MongoDB database using the CRUD module, and subsequently provide a user-friendly
# dashboard interface for the user to interact with the AAC database through a dashboard. The dashboard features a dash table, a geolocation map, 
# and three charts that interactively respond to the radio filter and the row selection. 
#
# Changes: I have added two additional graphs (bar and pie charts), dependencies were updated, hard-coded passwords are removed, and password handling is added. 
#
#Issues: none

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import Dash, dash_table, dcc, html, Input, Output
import plotly.express as px
from dash.dependencies import Input, Output
import base64
# Configure OS routines
import os
# Configure the plotting routines
import pandas as pd
import sys
import atexit

# import the CRUD module
from CRUD import CRUD

###########################
# Data Manipulation / Model
###########################
# get the environment passwords 
username = os.environ.get('DB_USER')
password = os.environ.get('DB_PASS')

# check the password is provided
if password is not None:
    # if the password is provided, pass the credentials as parameters and 
    # connect to database via CRUD Module
    db=CRUD(username, password)
# if the password is not provided, 
else: # print a message to the console
    print("Database password not set.")
    # terminate the application 
    sys.exit()

# class read method must support return of list object and accept projection json input
# sending the read method with no parameters requests all documents
df = pd.DataFrame.from_records(db.read({}))
        
# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the data frame here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dat frame that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug Statement
# print(len(df.to_dict(orient='records')))
# print(df.columns)

# define the common queries / filters for our dataset
# Define a water rescue query: It includes intact female dogs that belong to the breeds: lab, chesapeake bay retriever, 
# and newfoundland between 26-156 weeks.
water_rescue_query = {
    "animal_type": "Dog",
    "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
    "sex_upon_outcome": "Intact Female",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}    

# Define mountain rescue query: It includes intact male dogs that belong to the breeds: german shepherd, alaskan malamute, old english sheepdog,
# siberian husky, and rottweiler between 26-156 weeks.
mountain_rescue_query = {
    "animal_type": "Dog",
    "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
}

# Define disaster query: it includes intact male dogs that belong to the breeds doberman pinscher, german shepherd, golden retriever, 
# bloodhound, and rottweiler that are between 20-300 weeks.
disaster_query = {
    "animal_type": "Dog",
    "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
    "sex_upon_outcome": "Intact Male",
    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
}

# Function to mask confidential fields
def mask_field(name):
    return '********'

# Mask the confidential name field
df['name'] = df['name'].apply(mask_field)    

#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

# Note: Since may need the transparency provided by the PNG going forward, I kept the image as-is
image_filename = 'logo.png' 
# use with open to properly deallocate resources when finished with the image
with open('logo.png', 'rb') as file:
    encoded_image = base64.b64encode(file.read())

# define the app layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Animal Dashboard'))),
    html.Hr(),

    # Logo is top and centered 
    html.Div([  
        html.A(
            html.Center(html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                id='logoImage',
                style={'width': '10%'}  # scale the image down in size
            )),
            href='https://www.snhu.edu/'  # the link directs to the SNHU website
        ),
    ]),

    # Format the radio buttons and place under the logo
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water', 'value': 'water'},
                {'label': 'Mountain', 'value': 'mountain'},
                {'label': 'Disaster', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'},
            ],
            value='reset',
            labelStyle={'display': 'inline-block'}
        )
    ]),
    html.Hr(),

    # Format the data table, following the radio filters
    dash_table.DataTable(
        id='datatable-id',
        # define the columns 
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns], 
        # define the conditional styling
        style_data_conditional=[{'if': {'column_editable': False},'backgroundColor': 'white','color': 'rgb(30, 30, 30)'}],
        style_header_conditional=[{'if': {'column_editable': False},'backgroundColor': 'white','color': 'rgb(30, 30, 30)'}],
        data=df.to_dict('records'),  # convert the data frame to a dictionary 
        editable=False,  # set the data table to non-editable
        filter_action="native",
        sort_action="native", 
        sort_mode="multi",   # multi-column sorting
        column_selectable=False,  # disable the option to select columns
        row_selectable="single",  # one row selected at once
        row_deletable=False,   # rows should not be deleted
        selected_columns=[],  # initialize no columns selected 
        selected_rows=[0],
        derived_virtual_selected_rows=[], # initialize no virtual selected rows 
        page_action="native",  # native pagination
        page_current=0,
        page_size=10, # limit the records on one page to 10
       # derived_virtual_data=df.to_dict('records'),
        style_header={'border': '1px solid pink'},
    ),
    html.Br(),
    html.Hr(),
    
    
    #set up the chart and your geolocation chart side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        # add the scatter chart
        html.Div(
            id='graph-id',  # scatter chart to show the breed by count distribution
            className='col s12 m6',
            style={'width': '50%'} # set the width to half the page size
        ),
        # add the geolocation chart
        html.Div(
            id='map-id', # geolocation to display longitude / latitude coordinates of the selection
            className='col s12 m6'
        )
    ]),
    # below, set up the pie and bar chart side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        # add the bar chart
        html.Div(
            id='bar-id', # bar chart for outcome type by count
            className='col s12 m6',
            style={'width': '50%'}  # set the width to half the page size
        ),
        # add the pie chart
        html.Div(
            id='pie-id', # pie chart for age upon outcome by count
            className='col s12 m6',
            style={'width': '50%'}  # set the width to half the page size
        ) 
    ]),
    
    html.Hr(),
    # Developer name at the bottom
    html.Center(html.B(html.H1('Developed by Shera Adams'), style={'font-size': '10px'})),
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id', 'data'),
              Output('datatable-id', 'columns'),
              [Input('filter-type', 'value')])
def update_dashboard(filterValue, **kwargs):
    # Filter data based on the selected type 
    if filterValue == 'mountain':
        # save the df as result of the read function call passing the mountain query as a parameter
        df = pd.DataFrame.from_records(db.read(mountain_rescue_query))
    elif filterValue == 'water':
        # save the df as result of the read function call passing the water query as a parameter
        df = pd.DataFrame.from_records(db.read(water_rescue_query))
    elif filterValue == 'disaster':
        # save the df as result of the read function call passing the disaster query as a parameter
        df = pd.DataFrame.from_records(db.read(disaster_query))
    else:
        # return all data if reset (default data table/ read all data)
        df = pd.DataFrame.from_records(db.read({'animal_type': {'$in': ["Dog", "Cat", "Bird", "Other"]}}))
    
    # delete the _id column
    df.drop(columns=['_id'],inplace=True)   
        # mask name field
    df['name'] = df['name'].apply(mask_field)
    # define the column properties
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    # create a dictionary from the data frame
    data = df.to_dict('records')
    #return a tuple of data, columns
    return (data, columns)

#############################################
# DATA CHART 
#############################################
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)
def update_graphs(viewData, filterValue, **kwargs):
    # Create a data frame from the data
    dff = pd.DataFrame(viewData)
    # if the breed column is found
    if 'breed' in dff.columns:
        dff = dff['breed'].value_counts().reset_index()
        # define the column names as breed and count
        dff.columns = ['Breed', 'Count']

        # Combine the breeds with less than 70 members to make the chart more readable
        if filterValue == 'reset':
            dff.loc[dff['Count'] < 70, 'Breed'] = 'Other Breeds'

        # Create a bubble chart
        bubble_fig = px.scatter(
            dff,
            x='Breed',  # X-axis variable
            y='Count',  # Y-axis variable
            size='Count',  # Size of the bubbles
            color='Breed',  # Color of the bubbles
            title='Preferred Animals by Breed',  # title the chart
            template='plotly_white'  # set the template style
        )

        # set labels to diagonal 
        bubble_fig.update_layout(xaxis_tickangle=45)

        # Return the graph
        return [dcc.Graph(
            figure=bubble_fig,
            # set the size of the graph
            style={'width': '900px', 'height': '500px'}
        )]
    # if the breed column is not found, return an empty graph
    else:
        empty_fig = px.scatter(title='No Data Available', template='plotly_white')
        return [dcc.Graph(figure=empty_fig)]
        

#############################################
# PIE CHART: AGE UPON OUTCOME DISTRIBUTION
#############################################
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('pie-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')])
def update_pie(viewData, filterValue, **kwargs):
    # Create a data frame from the data
    dff = pd.DataFrame(viewData)
    # check that age upon outcome is in the data
    if 'age_upon_outcome_in_weeks' in dff.columns:
        # Group ages into six categories
        categories= [0, 4, 12, 26, 52, 100, 200, 500, float('inf')]
        # define the categories
        labels = ['0-4 Weeks', '4-12 Weeks', '12-26 Weeks', '26-52 Weeks', 
                  '52-100 weeks', '100-200 Weeks', '200-500 Weeks','>500 Weeks']
        # Create a new  age group column
        dff['age_group'] = pd.cut(dff['age_upon_outcome_in_weeks'], 
                                  bins=categories, labels=labels, right=False) 
        # count the members in each column
        age_counts = dff['age_group'].value_counts().reset_index()
        # rename the column
        age_counts.columns = ['Age Group', 'Count']
        
        # Create a pie chart
        pie_fig = px.pie(
            age_counts, 
            names='Age Group',   # Label for the pie chart
            values='Count',      # Values for each slice
            title='Preferred Animals by Age Upon Outcome',  # Title of the chart
            template='simple_white',       # Choose the template
        )
        pie_fig.update_layout(
            legend_title_text='Age Categories'
        )

        # Return the graph
        return [dcc.Graph(figure=pie_fig)]
    # if age upon outcome is not in the data, return an empty graph
    else:
        empty_fig = px.pie(title='No Data Available', template='plotly_white')
        return [dcc.Graph(figure=empty_fig)]
        
#############################################
# BAR CHART: OUTCOME DISTRIBUTION
#############################################
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('bar-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)
def update_bar(viewData, filterValue, **kwargs):
    # Create a data frame from the data
    dff = pd.DataFrame(viewData)
    # if outcome type is in the data
    if 'outcome_type' in dff.columns:
        dff = dff['outcome_type'].value_counts().reset_index()
        # rename the columns and define the df
        dff.columns = ['Outcome Type', 'Count']
        # Create a bar chart
        bar_fig = px.bar(
            dff,
            x='Outcome Type',      # X-axis variable
            y='Count',             # Y-axis variable
            color='Outcome Type',  # Color of the bars 
            title='Preferred Animals by Outcome Type',  # Title the chart
            template='plotly_white'  # Set the template style
        )
        # set labels to diagonal 
        bar_fig.update_layout(xaxis_tickangle=45)

        # Return the graph
        return [dcc.Graph(figure=bar_fig)]
    # if age upon outcome is not in the data return an empty graph
    else:
        empty_fig = px.bar(title='No Data Available', template='plotly_white')
        return [dcc.Graph(figure=empty_fig)]

#############################################
# update row style
#############################################
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]  
)
def update_styles(selected_rows, **kwargs):
    styles = [{
        'if': { 'row_index': i },  # Use 'row_index' to target rows
        'background_color': '#D2F3FF' 
    } for i in selected_rows]

#############################################
# Geolocation and marker
#############################################
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index, **kwargs):  
    # handle the case that the viewData is empty
    if viewData is None or not viewData:
        return []

    # Check for empty index
    if index is None or not index:
        return []
    
    # set the index default to the first row
    else: 
        row = index[0]  
    # define the data frame
    dff = pd.DataFrame.from_dict(viewData)
    # return the map with a marker at the index's longitude/ longitude coordinates
    return [     # Austin TX is at [30.75,-97.48]
        dl.Map(style={'width': '900px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),  
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]
# Cleanup function to close the database when finished
def cleanup():
    db.close()

# call the cleanup function when the user closes the browser
atexit.register(cleanup)

# Start the server and make the dashboard continuously available until the user closes the window
if __name__ == '__main__':
    app.run_server(debug=True, port=8081)



connection successful
